<a href="https://colab.research.google.com/github/popescuaaa/playground/blob/master/Simple_GAN_Improvement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as data

# Setup

In [ ]:
config = {
  'learning_rate': 0.1,

  'g': {
      'dim_latent': 128, # z dimension for generator
      'dim_hidden': 128, # size of the hidden layer
  },
}

## Helpers

# Generator

In [ ]:
# TODO: norm layer why ?

class Generator(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    __module_list = [
        nn.Linear(cfg['dim_latent'], cfg['dim_hidden']),
        nn.ReLU(),
        nn.Linear(cfg['dim_hidden'], cfg['dim_hidden'] * 2),
        nn.ReLU(),
        nn.Linear(cfg['dim_hidden'] * 2,  28 * 28), # size of a mnist sample
        nn.ReLU()
    ]

    # like a running container => in sequence
    self.__net = nn.Sequential(*__module_list)

  def forward(self, x):
     return self.__net(x)

# Discriminator

In [ ]:
class Discriminator(nn.Module):
  def __init__(self, dim_input, dim_output):
    super().__init__()
    __module_list = [
        nn.Linear(dim_input, dim_input // 2),
        nn.ReLU(),
        nn.Linear(dim_input // 2, dim_input // 4),
        nn.ReLU(),
        nn.Linear(dim_input // 4, dim_output),
        nn.Sigmoid() # Must be signmoid or smth that returns a vlue in [0, 1] to
                    # make the BCELoss work
    ]

    self.__net = nn.Sequential(*__module_list)

  def forward(self, x):
     return self.__net(x)

# System training

## Parameters and data

In [ ]:
env_learning_rate = config['learning_rate']

g_cfg = config['g']

g = Generator(g_cfg)
d = Discriminator(28 * 28, 1) # we must generate a score

## Data

In [ ]:
batch_size = 32
transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)),]
)

dataset = datasets.MNIST(root="dataset/", transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
optimizer_G = optim.Adam(g.parameters(), lr=config['learning_rate'])
optimizer_D = optim.Adam(d.parameters(), lr=config['learning_rate'])

criterion = nn.BCEWithLogitsLoss()

In [ ]:
def trainD(real, noise):
  fake_data = g(noise)
  real_data = real
  
  d_real = d(real_data).view(-1)
  d_fake = d(fake_data).view(-1)

  loss_fake = criterion(d_fake, torch.zeros_like(d_fake))
  loss_real = criterion(d_real, torch.ones_like(d_real))

  loss = 0.5 * loss_fake + 0.5 * loss_fake
  d.zero_grad()

  loss.backward()
  optimizer_D.step()

  return loss

def trainG(noise):
  fake_data = g(noise)
  output = d(fake_data).view(-1)

  loss = criterion(output, torch.ones_like(output))
  g.zero_grad()

  loss.backward()
  optimizer_G.step()

  return loss


def train_system():
  d_steps = 10
  g_steps = 10

  
  #for _ in range(g_steps):
  loss_g = trainG(noise)

  #for _ in range(d_steps): 
  loss_d = trainD(real, noise)

  return loss_d, loss_g

## Training

In [ ]:
num_epochs = 3
torch.random.manual_seed(42)
for epoch in range(num_epochs):
  for batch_idx, (real, _) in enumerate(loader):

    real = real.view(-1, 784) # reshape -1 => don't know how many rows but should 
                              # be 784 columns
    batch_size = real.shape[0]

    ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
    noise = torch.randn(batch_size, config['g']['dim_latent'])
    
    lossD, lossG = train_system()

    if batch_idx == 500:

        print(
            f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
                  Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
        )


## Result


```
Epoch [0/10] Batch 0/1875                   Loss D: 0.6815, loss G: 0.0000
Epoch [1/10] Batch 0/1875                   Loss D: 50.0000, loss G: 0.0000
Epoch [2/10] Batch 0/1875                   Loss D: 50.0000, loss G: 0.0000
Epoch [3/10] Batch 0/1875                   Loss D: 50.0000, loss G: 0.0000
Epoch [4/10] Batch 0/1875                   Loss D: 50.0000, loss G: 0.0000
Epoch [5/10] Batch 0/1875                   Loss D: 50.0000, loss G: 0.0000
```

The 1/2 = 50% is the convergence value for D and because we have loss at this value at epoch two the G can't learn properly.

